In [1]:
import os
os.chdir('../..')

In [2]:
from classes.matching import Matching
from classes.helpers import *
import numpy as np
import json
import pandas as pd
from datetime import datetime
import gzip

import ast

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

data_folder = '../data/'

In [3]:
ts = pd.read_csv(data_folder + 'tmp/time_series_1_valid.csv', header=[0,1])

# Transform strings into arrays
for i in ts.index:
    for key1 in ['ba', 'rb']:
        for key2 in ['dates', 'ratings', 'z_scores']:
            arr = ast.literal_eval(ts.loc[i][key1][key2])
            ts.set_value(i, (key1, key2), arr)

df = pd.read_csv(data_folder + 'matched/beers.csv', header=[0,1])

brews = {'ba': None, 'rb': None}
brew['ba'] = pd.read_csv(data_folder + 'ba/breweries.csv')
brew['rb'] = pd.read_csv(data_folder + 'rb/breweries.csv')

In [11]:
diffs = {'ba': [], 'rb': []}
for i in ts.index:
    row = ts.iloc[i]
    for key in ['rb', 'ba']:
        diffs[key].append(row[key]['z_scores'][0])
        
thresholds = {}
for key in ['ba', 'rb']:
    thresholds[key] = {}
    thresholds[key]['low'] = np.percentile(diffs[key], 15)
    thresholds[key]['high'] = np.percentile(diffs[key], 85)
    
for key in ['ba', 'rb']:
    tmp = []
    nbr_rats = []
    
    for i in ts.index:      
        if diffs[key][i] > thresholds[key]['high']:
            tmp.append('H')
        elif diffs[key][i] >= thresholds[key]['low']:
            tmp.append('M')
        else:
            tmp.append('L')
            
        nbr_rats.append(len(ts.loc[i][key]['ratings']))

    ts.loc[:, ((key, 'class'))] = tmp
    ts.loc[:, ((key, 'nbr_ratings'))] = nbr_rats

In [49]:
dfs = {'ba': None, 'rb': None}

for key in dfs.keys():
    dfs[key] = ts[key][['beer_id', 'class']]

    dfs[key]['style'] = np.array(df[df[key]['beer_id'].isin(dfs[key]['beer_id'])][key]['style'])
    dfs[key]['brewery_name'] = np.array(df[df[key]['beer_id'].isin(dfs[key]['beer_id'])][key]['brewery_name'])

    locations = []
    for i in dfs[key].index:
        brew_name = dfs[key].loc[i]['brewery_name']

        subdf = brews[key][brews[key]['name'] == brew_name]['location']

        loc = subdf.loc[subdf.index[0]]

        if 'United States' in loc:
            loc = 'United States'

        locations.append(loc)

    dfs[key]['location'] = locations
    
dfs['rb']['style'] = dfs['ba']['style']

# Style

In [149]:
styles = dfs['ba']['style'].unique()

In [150]:
df_json = {'ba': {},
           'rb': {}}

classes = ['H-L', 'L-H', 'H-M', 'M-H', 'M-L', 'L-M']

for key in df_json.keys():
    for cl in classes:
        df_json[key][cl] = []

In [151]:
for cl in classes:
    idx = np.array((dfs['ba']['class'] == cl[0]) & (dfs['rb']['class'] == cl[-1]))
    
    for key in df_json.keys():
        subdf = dfs[key][idx]
        
        if key == 'rb':
            cl = cl[::-1]
            
        df_json[key][cl].append(len(subdf))
        
        for s in styles:
            subsubdf = subdf[subdf['style'] == s]
            
            df_json[key][cl].append(len(subsubdf))

In [152]:
styles = np.insert(styles, 0, 'Total')

In [153]:
df_json['styles'] = {}
df_json['styles']['-'] = styles

In [154]:
df_styles = pd.DataFrame.from_dict({(i, j): df_json[i][j] 
                                    for i in df_json.keys() 
                                    for j in df_json[i].keys()})

In [155]:
df_styles.to_csv(data_folder + 'tmp/styles.csv', index=False)

# Country

In [170]:
countries = dfs['ba']['location'].unique()

In [171]:
df_json = {'ba': {},
           'rb': {}}

classes = ['H-L', 'L-H', 'H-M', 'M-H', 'M-L', 'L-M']

for key in df_json.keys():
    for cl in classes:
        df_json[key][cl] = []

In [172]:
for cl in classes:
    idx = np.array((dfs['ba']['class'] == cl[0]) & (dfs['rb']['class'] == cl[-1]))
    
    for key in df_json.keys():
        subdf = dfs[key][idx]
        
        if key == 'rb':
            cl = cl[::-1]
            
        df_json[key][cl].append(len(subdf))
        
        for c in countries:
            subsubdf = subdf[subdf['location'] == c]
            
            df_json[key][cl].append(len(subsubdf))

In [173]:
countries = np.insert(countries, 0, 'Total')

In [174]:
df_json['countries'] = {}
df_json['countries']['-'] = countries

In [175]:
df_countries = pd.DataFrame.from_dict({(i, j): df_json[i][j] 
                                       for i in df_json.keys() 
                                       for j in df_json[i].keys()})

In [178]:
df_countries.to_csv(data_folder + 'tmp/countries.csv', index=False)

In [179]:
df_countries

ba                                              countries   rb        \
    H-L   H-M  L-H   L-M   M-H   M-L                         -  H-L   H-M   
0   300  2499  234  2529  2560  2451                     Total  234  2560   
1     0     2    0     3     0     2          Northern Ireland    0     0   
2     1     2    1    14     9    12                     Wales    1     9   
3     0     2    2    36    19    15                  Scotland    2    19   
4    11    76   15   142   101   134                   England   15   101   
5     0     0    0     1     0     0                 Singapore    0     0   
6     1     0    0     1     1     1                     China    0     1   
7     0     0    0     0     0     1               Saint Lucia    0     0   
8     1     0    3     1    10     3                   Romania    3    10   
9     0     1    0     2     0     1               South Korea    0     0   
10    0     0    0     0     0     1                   Georgia    0     0   
11    0     0    0     1     0     1                 Hong Kong    0     0   
12    1     2    0     8     1     7                    Mexico    0     1   
13    0     0    0     1     0     1                      Laos    0     0   
14    0     0    0     0     0     0                 Indonesia    0     0   
15    0     0    0     3     0     2                   Ukraine    0     0   
16   16    93   14   157   142   298                    Canada   14   142   
17    0     6    0     4     0     1                  Portugal    0     0   
18    0     0    0     1     1     4                     India    0     1   
19    0     0    0     0     0     1               Puerto Rico    0     0   
20    1     5    1    18     6    10                     Japan    1     6   
21    0     0    1     7     3     3                   Hungary    1     3   
22    0     0    0     3     2     0                  Bulgaria    0     2   
23    0     0    0     0     0     0                      Togo    0     0   
24    0     2    0     1     0     1                   Croatia    0     0   
25    0     0    0     3     1     3                 Lithuania    0     1   
26    0     0    0     0     1     2                    Cyprus    0     1   
27    1    11   10   103    27    27                     Italy   10    27   
28    0     0    0     0     0     0  Turks and Caicos Islands    0     0   
29    0     0    0     0     0     1                  Mongolia    0     0   
..  ...   ...  ...   ...   ...   ...                       ...  ...   ...   
68    0     0    0     1     1     4                    Serbia    0     1   
69    0     0    0     0     0     1                 Macedonia    0     0   
70    0     0    1     1     2     1                    Latvia    1     2   
71    0     0    0     0     0     1                 Guatemala    0     0   
72    0     0    0     0     0     0                      Cuba    0     0   
73    0     0    0    18     1     9                   Finland    0     1   
74    4    25    9    96    37    31                    Sweden    9    37   
75    1     2    4    19    16    17                    Poland    4    16   
76    0     0    0     0     0     0        Cape Verde Islands    0     0   
77    0     0    0     1     0     0               Isle of Man    0     0   
78    5    24    0     5    10    13                   Ireland    0    10   
79    0     0    0     0     0     0                   Myanmar    0     0   
80    1     6    0     8     3     9                    Norway    0     3   
81  208  2022  123  1362  1856  1387             United States  123  1856   
82    0     0    0     0     0     3                Costa Rica    0     0   
83    0     0    0     0     0     0               North Korea    0     0   
84    1     0    0     1     0     0    Bosnia and Herzegovina    0     0   
85    0     0    0     0     0     0                   Lebanon    0     0   
86    0     0    0     0     0     0     Virgin Islands (U.S.)    0     0   
8